#### 1. Import required libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
import re

#### 2. Data Preparation

In [2]:
data = ["We are about to study the idea of a computational process",
"Computational processes are abstract beings that inhabit computers",
"As they evolve processes manipulate other abstract things called data",
"The evolution of a process is directed by a pattern of rules called a program", 
"People create programs to direct processes", 
"In effect we conjure the spirits of the computer with our spells"]

In [3]:
# clean data
clean_sentences = []
for sentence in data:
    # skip empty string
    if sentence == "":
        continue
    # remove special characters
    # sentence = re.sub('[^A-Za-z0-9]+', ' ', sentence)
    # # remove 1 letter words
    # sentence = re.sub(r'(?:^| )\w(?:$| )', ' ', sentence).strip()
    # # lower all characters
    sentence = sentence.lower()
    clean_sentences.append(sentence)

In [4]:
clean_sentences

['we are about to study the idea of a computational process',
 'computational processes are abstract beings that inhabit computers',
 'as they evolve processes manipulate other abstract things called data',
 'the evolution of a process is directed by a pattern of rules called a program',
 'people create programs to direct processes',
 'in effect we conjure the spirits of the computer with our spells']

In [5]:
corpus = clean_sentences

In [6]:
# Convert the corpus to a sequence of integers
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(corpus)
# sequences = tokenizer.texts_to_sequences(corpus)
# print("After converting our words in the corpus into vector of integers:")
# print(sequences)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
print(sequences)

[[5, 6, 12, 7, 13, 1, 14, 2, 3, 8, 9], [8, 4, 6, 10, 15, 16, 17, 18], [19, 20, 21, 4, 22, 23, 10, 24, 11, 25], [1, 26, 2, 3, 9, 27, 28, 29, 3, 30, 2, 31, 11, 3, 32], [33, 34, 35, 7, 36, 4], [37, 38, 5, 39, 1, 40, 2, 1, 41, 42, 43, 44]]


In [13]:
# creating dictionary for word to index and index to word
index_to_word_map = {}
word_to_index_map = {}

for index_1, sequence in enumerate(sequences):
    print(sequence)
    words_in_sentence = clean_sentences[index_1].split()
    print(words_in_sentence)
    for index_2, value in enumerate(sequence):
        index_to_word_map[value] = words_in_sentence[index_2]
        word_to_index_map[words_in_sentence[index_2]] = value

# id2w = {}
# w2id = {}

# for index_1, sequence in enumerate(sequences):
#     words_in_sentence = clean_sentences[index_1].split()
#     for index_2, value in enumerate(sequence):
#         id2w[value] = words_in_sentence[index_2]
#         w2id[words_in_sentence[index_2]] = value

[5, 6, 12, 7, 13, 1, 14, 2, 3, 8, 9]
['we', 'are', 'about', 'to', 'study', 'the', 'idea', 'of', 'a', 'computational', 'process']
[8, 4, 6, 10, 15, 16, 17, 18]
['computational', 'processes', 'are', 'abstract', 'beings', 'that', 'inhabit', 'computers']
[19, 20, 21, 4, 22, 23, 10, 24, 11, 25]
['as', 'they', 'evolve', 'processes', 'manipulate', 'other', 'abstract', 'things', 'called', 'data']
[1, 26, 2, 3, 9, 27, 28, 29, 3, 30, 2, 31, 11, 3, 32]
['the', 'evolution', 'of', 'a', 'process', 'is', 'directed', 'by', 'a', 'pattern', 'of', 'rules', 'called', 'a', 'program']
[33, 34, 35, 7, 36, 4]
['people', 'create', 'programs', 'to', 'direct', 'processes']
[37, 38, 5, 39, 1, 40, 2, 1, 41, 42, 43, 44]
['in', 'effect', 'we', 'conjure', 'the', 'spirits', 'of', 'the', 'computer', 'with', 'our', 'spells']


In [14]:
print(index_to_word_map)
print('\n')
print(word_to_index_map)

{5: 'we', 6: 'are', 12: 'about', 7: 'to', 13: 'study', 1: 'the', 14: 'idea', 2: 'of', 3: 'a', 8: 'computational', 9: 'process', 4: 'processes', 10: 'abstract', 15: 'beings', 16: 'that', 17: 'inhabit', 18: 'computers', 19: 'as', 20: 'they', 21: 'evolve', 22: 'manipulate', 23: 'other', 24: 'things', 11: 'called', 25: 'data', 26: 'evolution', 27: 'is', 28: 'directed', 29: 'by', 30: 'pattern', 31: 'rules', 32: 'program', 33: 'people', 34: 'create', 35: 'programs', 36: 'direct', 37: 'in', 38: 'effect', 39: 'conjure', 40: 'spirits', 41: 'computer', 42: 'with', 43: 'our', 44: 'spells'}


{'we': 5, 'are': 6, 'about': 12, 'to': 7, 'study': 13, 'the': 1, 'idea': 14, 'of': 2, 'a': 3, 'computational': 8, 'process': 9, 'processes': 4, 'abstract': 10, 'beings': 15, 'that': 16, 'inhabit': 17, 'computers': 18, 'as': 19, 'they': 20, 'evolve': 21, 'manipulate': 22, 'other': 23, 'things': 24, 'called': 11, 'data': 25, 'evolution': 26, 'is': 27, 'directed': 28, 'by': 29, 'pattern': 30, 'rules': 31, 'progr

#### 3. Generate Training data

In [15]:
# Define the parameters
vocab_size = len(tokenizer.word_index)+1
embedding_size = 10
window_size = 2

contexts = []
targets = []

for sequence in sequences:
    for i in range(window_size, len(sequence)-window_size):
        context = sequence[i - window_size: i] + sequence[i + 1:i + window_size + 1]
        target = sequence[i]
        contexts.append(context)
        targets.append(target)

In [16]:
# Sample of training data
for i in range(5):
    words = []
    target = index_to_word_map.get(targets[i])
    for j in contexts[i]:
        words.append(index_to_word_map.get(j))
    print(words, "=>", target)

['we', 'are', 'to', 'study'] => about
['are', 'about', 'study', 'the'] => to
['about', 'to', 'the', 'idea'] => study
['to', 'study', 'idea', 'of'] => the
['study', 'the', 'of', 'a'] => idea


In [17]:
# Convert the contexts and targets to numpy arrays
X = np.array(contexts)
Y = np.array(targets)

#### 4. Training the Model

In [18]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, 
output_dim=embedding_size, input_length=2 * window_size))

model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(units=vocab_size, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 10)             450       
                                                                 
 lambda (Lambda)             (None, 10)                0         
                                                                 
 dense (Dense)               (None, 256)               2816      
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 dense_2 (Dense)             (None, 45)                23085     
                                                                 
Total params: 157,935
Trainable params: 157,935
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, Y, epochs=200, verbose=1)

Epoch 1/200
2/2 [==============================] - 1s 21ms/step - loss: 3.8070 - accuracy: 0.0263
Epoch 2/200
2/2 [==============================] - 0s 10ms/step - loss: 3.7984 - accuracy: 0.1053
Epoch 3/200
2/2 [==============================] - 0s 16ms/step - loss: 3.7912 - accuracy: 0.1579
Epoch 4/200
2/2 [==============================] - 0s 16ms/step - loss: 3.7832 - accuracy: 0.1842
Epoch 5/200
2/2 [==============================] - 0s 11ms/step - loss: 3.7724 - accuracy: 0.1842
Epoch 6/200
2/2 [==============================] - 0s 10ms/step - loss: 3.7595 - accuracy: 0.1579
Epoch 7/200
2/2 [==============================] - 0s 16ms/step - loss: 3.7428 - accuracy: 0.1579
Epoch 8/200
2/2 [==============================] - 0s 16ms/step - loss: 3.7226 - accuracy: 0.1579
Epoch 9/200
2/2 [==============================] - 0s 13ms/step - loss: 3.6971 - accuracy: 0.1579
Epoch 10/200
2/2 [==============================] - 0s 10ms/step - loss: 3.6663 - accuracy: 0.1053
Epoch 11/200
2/2 [=

2/2 [==============================] - 0s 8ms/step - loss: 0.6701 - accuracy: 0.8421
Epoch 85/200
2/2 [==============================] - 0s 8ms/step - loss: 0.6518 - accuracy: 0.8421
Epoch 86/200
2/2 [==============================] - 0s 5ms/step - loss: 0.6277 - accuracy: 0.8421
Epoch 87/200
2/2 [==============================] - 0s 8ms/step - loss: 0.6018 - accuracy: 0.8421
Epoch 88/200
2/2 [==============================] - 0s 6ms/step - loss: 0.5858 - accuracy: 0.8421
Epoch 89/200
2/2 [==============================] - 0s 4ms/step - loss: 0.5662 - accuracy: 0.8684
Epoch 90/200
2/2 [==============================] - 0s 8ms/step - loss: 0.5460 - accuracy: 0.8947
Epoch 91/200
2/2 [==============================] - 0s 8ms/step - loss: 0.5394 - accuracy: 0.8684
Epoch 92/200
2/2 [==============================] - 0s 4ms/step - loss: 0.5208 - accuracy: 0.8684
Epoch 93/200
2/2 [==============================] - 0s 8ms/step - loss: 0.4884 - accuracy: 0.8947
Epoch 94/200
2/2 [===============

2/2 [==============================] - 0s 8ms/step - loss: 0.0390 - accuracy: 1.0000
Epoch 167/200
2/2 [==============================] - 0s 9ms/step - loss: 0.0378 - accuracy: 1.0000
Epoch 168/200
2/2 [==============================] - 0s 9ms/step - loss: 0.0364 - accuracy: 1.0000
Epoch 169/200
2/2 [==============================] - 0s 9ms/step - loss: 0.0350 - accuracy: 1.0000
Epoch 170/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0341 - accuracy: 1.0000
Epoch 171/200
2/2 [==============================] - 0s 12ms/step - loss: 0.0333 - accuracy: 1.0000
Epoch 172/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0322 - accuracy: 1.0000
Epoch 173/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0315 - accuracy: 1.0000
Epoch 174/200
2/2 [==============================] - 0s 8ms/step - loss: 0.0305 - accuracy: 1.0000
Epoch 175/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0301 - accuracy: 1.0000
Epoch 176/200
2/2 [====

In [20]:
# Get the word embeddings
embeddings = model.get_weights()[0] # Weights of the model of first layer

#### 5. Output

In [21]:
# test model
test_sentences = [
    "we are to study",
    "create programs direct processes",
    "spirits process study program",
    "idea study people create"
]

#### 6. Predictions

In [22]:
for test_sentence in test_sentences:
    test_words = test_sentence.split(" ")
    print("Words: ", test_words)
    x_test = []
    for i in test_words:
        x_test.append(word_to_index_map.get(i))
    x_test = np.array([x_test])
    print("Indexs: ", x_test)
    test_predictions = model.predict(x_test)
    y_pred = np.argmax(test_predictions[0])
    print("Predictons: ",test_words, " => ", index_to_word_map.get(y_pred))
    print("\n")

Words:  ['we', 'are', 'to', 'study']
Indexs:  [[ 5  6  7 13]]
1/1 [==============================] - 0s 109ms/step
Predictons:  ['we', 'are', 'to', 'study']  =>  about


Words:  ['create', 'programs', 'direct', 'processes']
Indexs:  [[34 35 36  4]]
1/1 [==============================] - 0s 36ms/step
Predictons:  ['create', 'programs', 'direct', 'processes']  =>  to


Words:  ['spirits', 'process', 'study', 'program']
Indexs:  [[40  9 13 32]]
1/1 [==============================] - 0s 16ms/step
Predictons:  ['spirits', 'process', 'study', 'program']  =>  the


Words:  ['idea', 'study', 'people', 'create']
Indexs:  [[14 13 33 34]]
1/1 [==============================] - 0s 37ms/step
Predictons:  ['idea', 'study', 'people', 'create']  =>  programs


